In [93]:
from conllu import parse, parse_tree
from collections import Counter
import numpy as np
import sys
import random
import string

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, GRU, LSTM, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping


from tensorflow.keras.preprocessing import sequence

from tensorflow.keras import utils

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)

MIN_WORD_FREQUENCY = 2
SEQUENCE_LEN = 5
BATCH_SIZE = 32

Задача II : Генерация текстов
На лекции по рекуррентным нейронным сетям рассматривалась модель генерации текста с использованием символьной языковой модели.
Вам необходимо написать нейронную модель, предназначенную для генерации текста, но чтобы как базовые единицы использовались слова (а не символы).

После чего, необходимо подсчитать перплексию для языковой модели (https://en.wikipedia.org/wiki/Perplexity). Помните, что перплексия рассчитывается на новых текстах. 
То есть Вы возьмете какую-либо книгу (в электронном формате), и обучите на ней языковую модель. Затем возьмите другое произведение этого же автора и вычислите на ней перплексию.

Задача предполагает использование рекуррентной нейронной сети как основы, но с точки зрения архитектуры Вы свободны в плане выбора. Примените как LSTM, так и GRU и сравните их перплексию (и общий взгляд на адекватность генерируемого текста).
Вы можете добавить какие угодно (рабочие) модификации архитектуры, и описать их в сравнительном анализе, что даст дополнительные баллы.

**Имена файлов с тренировочным и тестовым наборами:**

In [94]:
corpus = "olesya.txt_Ascii.txt"

In [95]:
with open(corpus) as f:
    text = f.read().lower().replace('\n', ' \n ')
print('Corpus length in characters:', len(text))

text_in_words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print('Corpus length in words:', len(text_in_words))

Corpus length in characters: 145247
Corpus length in words: 23911


In [96]:
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = " ".join(doc)
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [97]:
# clean document
text_in_words = clean_doc(text_in_words)
print(text_in_words[:200])
print('Total Tokens: %d' % len(text_in_words))
print('Unique Tokens: %d' % len(set(text_in_words)))

['мой', 'слуга', 'повар', 'и', 'спутник', 'по', 'охоте', 'полесовщик', 'ярмола', 'вошел', 'в', 'комнату', 'согнувшись', 'под', 'вязанкой', 'дров', 'сбросил', 'ее', 'с', 'грохотом', 'на', 'пол', 'и', 'подышал', 'на', 'замерзшие', 'пальцы', 'у', 'какой', 'ветер', 'паныч', 'на', 'дворе', 'сказал', 'он', 'садясь', 'на', 'корточки', 'перед', 'заслонкой', 'нужно', 'хорошо', 'в', 'грубке', 'протопить', 'позвольте', 'запалочку', 'паныч', 'значит', 'завтра', 'на', 'зайцев', 'не', 'пойдем', 'а', 'как', 'ты', 'думаешь', 'ярмола', 'нет', 'не', 'можно', 'слышите', 'какая', 'завируха', 'заяц', 'теперь', 'лежит', 'и', 'а', 'ни', 'мурмур', 'завтра', 'и', 'одного', 'следа', 'не', 'увидите', 'судьба', 'забросила', 'меня', 'на', 'целых', 'шесть', 'месяцев', 'в', 'глухую', 'деревушку', 'волынской', 'губернии', 'на', 'окраину', 'полесья', 'и', 'охота', 'была', 'единственным', 'моим', 'занятием', 'и', 'удовольствием', 'признаюсь', 'в', 'то', 'время', 'когда', 'мне', 'предложили', 'ехать', 'в', 'деревню', 'я

*Посчитаем некоторые статистики*

In [98]:
# Calculate word frequency
word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(text_in_words)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

Unique words before ignoring: 7305
Ignoring words with frequency < 2
Unique words after ignoring: 2083


Можно наблюдать, что большая часть слов встречается в тексте всего 1 раз, но если мы удалим их из словаря, то получим неполноценный авторский словарь и бессмысленные предложения. Предлагается так же использовать последовательности длиной 2, чтобы сохранить большую часть авторских слов.

In [99]:
# cut the text in semi-redundant sequences of SEQUENCE_LEN words
STEP = 1
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 16782
Remaining sequences: 5335


In [100]:
sentences[3000:3005]

[['пол', 'если', 'в', 'такую', 'минуту'],
 ['ко', 'мне', 'свое', 'лицо', 'в'],
 ['моих', 'слов', 'иногда', 'мне', 'казалось'],
 ['слов', 'иногда', 'мне', 'казалось', 'что'],
 ['в', 'ней', 'всего', 'лишь', 'несколько']]

## Препроцессинг данных

In [103]:
def shuffle_and_split_training_set(sentences_original, labels_original, percentage_test=10):
    print('Shuffling sentences')
    tmp_sentences = []
    tmp_next_char = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_char.append(labels_original[i])
    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_char[:cut_index], tmp_next_char[cut_index:]

    print("Training set = %d\nTest set = %d" % (len(x_train), len(y_test)))
    return x_train, y_train, x_test, y_test

In [104]:
sentences, next_words, sentences_test, next_words_test = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Training set = 4320
Test set = 481


**Генератор для fit_generator():**

In [105]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index]]] = 1

            index = index + 1
            if index == len(sentence_list):
                index = 0
        yield x, y

## Используемые архитектуры:

### 1. Bidirectional LSTM

In [116]:
def generate_LSTM():
    model = Sequential()
    model.add(LSTM(128, input_shape=(SEQUENCE_LEN, len(words))))
    model.add(Dropout(0.2))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    optimizer = RMSprop(lr=0.01)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
    
    return model

### 2. GRU

In [117]:
def generate_GRU():
    model = Sequential()
    model.add(GRU(128, input_shape=(SEQUENCE_LEN, len(words))))
    model.add(Dropout(0.1))
    model.add(Dense(len(words)))
    model.add(Activation("softmax"))
    optimizer = RMSprop(lr=0.01)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
    return model

## Обучение

In [118]:
file_path = "./checkpoints/LSTM_text-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % (
    len(words),
    SEQUENCE_LEN,
    MIN_WORD_FREQUENCY
)
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, early_stopping]

In [119]:
model_LSTM = generate_LSTM()

model_LSTM.fit_generator(generator(sentences, next_words, BATCH_SIZE),
    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
    epochs=100,
    callbacks=callbacks_list,
    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),              validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Epoch 1/100
136/136 [==============================] - 14s 105ms/step - loss: 6.6342 - acc: 0.0365 - val_loss: 6.3559 - val_acc: 0.0527
Epoch 2/100
136/136 [==============================] - 8s 56ms/step - loss: 6.0079 - acc: 0.0441 - val_loss: 6.4668 - val_acc: 0.0527
Epoch 3/100
136/136 [==============================] - 8s 56ms/step - loss: 5.6160 - acc: 0.0579 - val_loss: 7.0288 - val_acc: 0.0527
Epoch 4/100
136/136 [==============================] - 8s 56ms/step - loss: 5.1661 - acc: 0.0919 - val_loss: 7.7286 - val_acc: 0.0488
Epoch 5/100
136/136 [==============================] - 8s 57ms/step - loss: 4.6987 - acc: 0.1540 - val_loss: 7.3091 - val_acc: 0.0430
Epoch 6/100
136/136 [==============================] - 8s 57ms/step - loss: 4.2331 - acc: 0.2305 - val_loss: 7.4125 - val_acc: 0.0371


In [56]:
file_path = "./checkpoints/GRU_text-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % (
    len(words),
    SEQUENCE_LEN,
    MIN_WORD_FREQUENCY
)
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, early_stopping]

In [57]:
model_LSTM = generate_GRU()

model_LSTM.fit_generator(generator(sentences, next_words, BATCH_SIZE),
    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
    epochs=100,
    callbacks=callbacks_list,
    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),              validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Epoch 1/100
239/239 [==============================] - 10s 43ms/step - loss: 6.3485 - acc: 0.0638 - val_loss: 6.2449 - val_acc: 0.0775
Epoch 2/100
239/239 [==============================] - 5s 23ms/step - loss: 5.5462 - acc: 0.1106 - val_loss: 6.3037 - val_acc: 0.0984
Epoch 3/100
239/239 [==============================] - 5s 22ms/step - loss: 4.9776 - acc: 0.1644 - val_loss: 6.6202 - val_acc: 0.1215
Epoch 4/100
239/239 [==============================] - 5s 22ms/step - loss: 4.5584 - acc: 0.2249 - val_loss: 6.9641 - val_acc: 0.1042
Epoch 5/100
239/239 [==============================] - 6s 24ms/step - loss: 4.2998 - acc: 0.2737 - val_loss: 7.1916 - val_acc: 0.0995
Epoch 6/100
239/239 [==============================] - 5s 23ms/step - loss: 4.1761 - acc: 0.3087 - val_loss: 7.2767 - val_acc: 0.0949
Epoch 7/100
239/239 [==============================] - 6s 24ms/step - loss: 4.0772 - acc: 0.3241 - val_loss: 7.3213 - val_acc: 0.0949
Epoch 8/100
239/239 [==============================] - 6s 24m